[Importing Excel spreadsheet](#Importing-Excel-spreadsheet)

<a id='Importing-Excel-spreadsheet'></a>
# Importing Excel spreadsheet

In [1]:
import pandas as pd
import random
df = pd.read_excel("To_add.xlsx")
#Visualize part of the dataset.
df.head()

,Microorganism,Tax ID,Plastic,Ref,Enzyme,Gene
0,Bacillus flexus,86664,PP,"Arkatkar, A., Arutchelvi, J., Bhaduri, S., Upp...",No,No
1,Penicillium chrysogenum,5076,PE,"Sangale, M. K., Shahnawaz, M., & Ade, A. B. (2...",No,No
2,Aspergillus sydowii,75750,PE,"Sangale, M. K., Shahnawaz, M., & Ade, A. B. (2...",No,No
3,Aspergillus niger,5061,PE,"Sangale, M. K., Shahnawaz, M., & Ade, A. B. (2...",No,No
4,Aspergillus awamori,105351,PE,"Sangale, M. K., Shahnawaz, M., & Ade, A. B. (2...",No,No


# Create dataframe with all microorganisms in the rows and the plastics in the columns

In [2]:
plastic_types = df['Plastic'].unique()
tax_ids = df['Tax ID'].unique()
df2 = pd.DataFrame(0, index=tax_ids, columns=plastic_types)
df2.sort_index(inplace=True)
df2.sort_index(axis=1, inplace=True)
df2.head()

,Ecoflex,HDPE,LDPE,Nylon 4,Nylon 6,Oxo-biodegradable PE,P(3HB-co-3HV),P(HB-co-HV),P3HB,P3HB/P3HV,...,PLA Blend,PLLA,PLLA Blend,PP,PS,PTC,PU,PVA-LLDPE,PVC,PVC Blend
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
287,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
303,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1270,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1390,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
for index, row in df.iterrows():
    plastic = row['Plastic']
    tax_id = row['Tax ID']
    df2.at[tax_id, plastic] = 1
df2.head()

,Ecoflex,HDPE,LDPE,Nylon 4,Nylon 6,Oxo-biodegradable PE,P(3HB-co-3HV),P(HB-co-HV),P3HB,P3HB/P3HV,...,PLA Blend,PLLA,PLLA Blend,PP,PS,PTC,PU,PVA-LLDPE,PVC,PVC Blend
0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
287,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
303,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1270,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1390,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Create id file to be imported by MEGAN

In [4]:
#Create a file with just the IDs to be imported by MEGAN.
ids = df2.index.values
df2.index.values.tofile("ids.csv", sep="\n")
ids[0] = 1
ids

array([      1,     287,     303,    1270,    1390,    1396,    1402,
          1408,    1409,    1423,    4929,    5061,    5076,    5078,
          5341,    5547,   28454,   29917,   33178,   33941,   35703,
         38033,   40324,   41061,   45462,   48296,   75750,   76759,
         86664,   94625,  105351,  106590,  169177,  180282,  181124,
        226011,  226117,  332410,  335286,  405444,  412384,  477689,
        570156,  587753, 1755431, 1755435, 1886637, 1898440])

# Setting up a local copy of the NCBI taxonomy database

In [ ]:
from ete3 import NCBITaxa
ncbi = NCBITaxa()
ncbi.update_taxonomy_database()

NCBI database not present yet (first time used?)
Done. Parsing...


Loading node names...


# Getting NCBI species tree topology

In [81]:
from ete3 import Tree, TreeStyle, TextFace, PhyloTree
t = ncbi.get_topology(ids, intermediate_nodes=True)
t.write(format=7, features=["sci_name"], outfile="tree.nw")
ts = TreeStyle()
ts.show_leaf_name = True
ts.mode = "c"
ts.arc_start = -180 # 0 degrees = 3 o'clock
ts.arc_span = 360
ts.title.add_face(TextFace("Plastic Degraders", fsize=40), column=0)
#t.show(tree_style=ts)


# Creating Itol dataset files

In [9]:
number_of_items = len(df.columns)
#colors = ("#2f285e","#525ca7","#244567","#002600","#f59a44","#dc143c","#e6c9eb","#e1bee7","#00d2ff","#3d7eff","#2e93b1","#5972b6","#dda9b4","#4169e1","#40e0d0","#dab420","#420dab","#696969","#e6c7dd","#941414","#b91919","#a31616","#676700","#737300","#8c8c19","#808000","#8f5d2b","#7b5025","#67431f","#3eb1c8","#2e93b1","#96c95c","#ffdeb4","#ffcc5c","#ffafbd","#f5b0a4","#15c3d3","#96c95c","#8bc34a","#ffdeb4","#2fc3e0","#3eb1c8","#2f285e","#525ca7","#244567","#002600","#f59a44","#dc143c","#e6c9eb","#e1bee7","#00d2ff","#3d7eff","#2e93b1","#5972b6","#dda9b4","#4169e1","#40e0d0","#dab420","#420dab","#696969","#e6c7dd","#941414","#b91919","#a31616","#676700","#737300","#8c8c19","#808000","#8f5d2b","#7b5025","#67431f","#3eb1c8","#2e93b1","#96c95c","#ffdeb4","#ffcc5c","#ffafbd","#f5b0a4","#15c3d3","#96c95c","#8bc34a","#ffdeb4","#2fc3e0","#3eb1c8","#2f285e","#525ca7","#244567","#002600","#f59a44","#dc143c","#e6c9eb","#e1bee7","#00d2ff","#3d7eff","#2e93b1","#5972b6","#dda9b4","#4169e1","#40e0d0","#dab420","#420dab","#696969","#e6c7dd","#941414","#b91919","#a31616","#676700","#737300","#8c8c19","#808000","#8f5d2b","#7b5025","#67431f","#3eb1c8","#2e93b1","#96c95c","#ffdeb4","#ffcc5c","#ffafbd","#f5b0a4","#15c3d3","#96c95c","#8bc34a","#ffdeb4","#2fc3e0","#3eb1c8","#2f285e","#525ca7","#244567","#002600","#f59a44","#dc143c","#e6c9eb","#e1bee7","#00d2ff","#3d7eff","#2e93b1","#5972b6","#dda9b4","#4169e1","#40e0d0","#dab420","#420dab","#696969","#e6c7dd","#941414","#b91919","#a31616","#676700","#737300","#8c8c19","#808000","#8f5d2b","#7b5025","#67431f","#3eb1c8","#2e93b1","#96c95c","#ffdeb4","#ffcc5c","#ffafbd","#f5b0a4","#15c3d3","#96c95c","#8bc34a","#ffdeb4","#2fc3e0","#3eb1c8","#2f285e","#525ca7","#244567","#002600","#f59a44","#dc143c","#e6c9eb","#e1bee7","#00d2ff","#3d7eff","#2e93b1","#5972b6","#dda9b4","#4169e1","#40e0d0","#dab420","#420dab","#696969","#e6c7dd","#941414","#b91919","#a31616","#676700","#737300","#8c8c19","#808000","#8f5d2b","#7b5025","#67431f","#3eb1c8","#2e93b1","#96c95c","#ffdeb4","#ffcc5c","#ffafbd","#f5b0a4","#15c3d3","#96c95c","#8bc34a","#ffdeb4","#2fc3e0","#3eb1c8","#2f285e","#525ca7","#244567","#002600","#f59a44","#dc143c","#e6c9eb","#e1bee7","#00d2ff","#3d7eff","#2e93b1","#5972b6","#dda9b4","#4169e1","#40e0d0","#dab420","#420dab","#696969","#e6c7dd","#941414","#b91919","#a31616","#676700","#737300","#8c8c19","#808000","#8f5d2b","#7b5025","#67431f","#3eb1c8","#2e93b1","#96c95c","#ffdeb4","#ffcc5c","#ffafbd","#f5b0a4","#15c3d3","#96c95c","#8bc34a","#ffdeb4","#2fc3e0","#3eb1c8")
r = lambda: random.randint(0,255)
print('#%02X%02X%02X' % (r(),r(),r()))
#table = list(df)
#table

#57EE8A


In [17]:
for column_number in range(2, number_of_items):
	f = open(table[0][column_number].rstrip()+".txt", "w")
	f.write("""DATASET_BINARY
#Binary datasets are visualized as filled or empty symbols, depending on the value associated with a node  or 1).
#Each node can have multiple associated values, and each value will be represented by a symbol (defined in FIELD_SHAPES) with corresponding color and label (from FIELD_COLORS and FIELD_LABELS).
#Possible values (defined under DATA below) for each node are 1 (filled shapes), (empty shapes) and -1 (completely omitted).

#lines starting with a hash are comments and ignored during parsing

#=================================================================#
#                    MANDATORY SETTINGS                           #
#=================================================================#
#select the separator which is used to delimit the data below (TAB,SPACE or COMMA).This separator must be used throughout this file (except in the SEPARATOR line, which uses space).
#SEPARATOR TAB
#SEPARATOR SPACE
SEPARATOR COMMA

#label is used in the legend table (can be changed later)
DATASET_LABEL,"""+table[0][column_number]+"""

#dataset color (can be changed later)
COLOR,"""+colors[column_number]+"""

#shapes for each field column; possible choices are
#1: rectangle 
#2: circle
#3: star
#4: right pointing triangle
#5: left pointing triangle
#6: check mark
FIELD_SHAPES,1

#field labels
FIELD_LABELS,f1

#=================================================================#
#                    OPTIONAL SETTINGS                            #
#=================================================================#

#define colors for each individual field column (if not defined all symbols will use the main dataset color, defined in COLOR above)
#FIELD_COLORS,#f0


#=================================================================#
#     all other optional settings can be set or changed later     #
#           in the web interface (under 'Datasets' tab)           #
#=================================================================#

#always show internal values; if set, values associated to internal nodes will be displayed even if these nodes are not collapsed. It could cause overlapping in the dataset display.
#SHOW_INTERNAL

#left margin, used to increase/decrease the spacing to the next dataset. Can be negative, causing datasets to overlap.
#MARGIN
#align symbols to the end of each leaf label (individual dataset field will not be aligned to each other)
#ALIGN_TO_LABELS,1

#symbol height factor; Default symbol height will be slightly less than the available space between leaves, but you can set a multiplication factor here to increase/decrease it (values from to 1 will decrease it, values above 1 will increase it)
#HEIGHT_FACTOR,1

#increase/decrease the spacing between individual levels, when there is more than one binary level defined 
#SYMBOL_SPACING,

#display or hide the text labels above each field column
#SHOW_LABELS,1

#Example dataset with 4 columns (circle, left triangle, right triangle and rectangle):
#FIELD_SHAPES,2,4,5,1
#FIELD_LABELS,f1,f2,f3,f4
#FIELD_COLORS,#f0,0f0,#fff0,0ff

#Internal tree nodes can be specified using IDs directly, or using the 'last common ancestor' method described in iTOL help pages
#=================================================================#
#       Actual data follows after the DATA keyword              #
#=================================================================#
DATA""")
	for line in table[1:]:
		#organism_name = line[0].replace(" ", "_").rstrip()
		#feature_binary = line[column_number]
		#f.write("\n"+organism_name+","+feature_binary)
		print(line)
	f.close()

NCBI ID
PVC
Nylon
Polyethylene (PE)
Polyurethane (PU)
PET
Polystyrene (PS)
Polypropylene (PP)
HDPE
LDPE
PHB
PCL
PHV
Poly(3HB-co-4HB)
PES
PBS
Sky-Green
PLA
Mater-Bi
PMA
BTA copolyester
P(3HB-co-3MP)
PHBV
P(3HB-co-3HP)
P3HP
P4HB
Polyether sulfone
LDPE containing pro-oxidant and 6% starch
LDPE Oxo-Biodegradable
poly(3-hydroxy-5-phenylvalerate) (PHPV)
Polyhydroxyoctanoate (PHO)
Polyhydroxynonanoate (PHN)
(Type of polymer) Addition
(Type of polymer) Condensation
(Class) Vinyl polymer
(Class) Polyamide
(Class) Polyester
(Class) Polysulfone
(Class) Polyurethane
(Sub-class) Aromatic hydrocarbon polymer 
(Sub-class) Chlorinated hydrocarbon polymer
(Sub-class) Polyethylene
(Sub-class) Polypropylene
(Sub-class) Aliphatic polyamide
(Sub-class) Aliphatic polyester
(Sub-class) Aliphatic-aromatic copolyester
(Sub-class) Aliphatic-aromatic polyester
 (Sub-class) Aromatic polyester
 (Sub-class) PHA
(Sub-class)  Aromatic polysulfone
(Sub-class)  Polyurethane
Polyurethanase_esterase_A_Pseudomonas_chloror

IndexError: string index out of range